In [3]:
#run check

In [20]:
import numpy as np
import random
import math
import os
import time

def read_tsp_file(filename):
    """
    Read TSP file and extract information.
    """
    with open(filename, 'r') as file:
        lines = file.readlines()
        num_locations = int(lines[0].strip())
        adjacency_matrix = np.zeros((num_locations, num_locations))
        for i in range(num_locations):
            distances = list(map(float, lines[i + 1].strip().split()))
            adjacency_matrix[i] = distances
    return num_locations, adjacency_matrix

def distance(city1, city2):
    """
    Calculate Euclidean distance between two cities.
    """
    return np.linalg.norm(city1 - city2)

def total_distance(cities_order, distances):
    """
    Calculate the total distance of the path.
    """
    total = 0
    num_cities = len(cities_order)
    for i in range(num_cities - 1):
        total += distances[cities_order[i], cities_order[i + 1]]
    total += distances[cities_order[-1], cities_order[0]]  # Return to the starting city
    return total

def k_opt_swap(route, k):
    """
    Perform a k-opt swap on the route.
    """
    num_cities = len(route)
    i, j = sorted(random.sample(range(num_cities), 2))
    if j - i < k:
        i, j = i, i + k
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    return new_route

def two_opt_swap(route):
    """
    Perform a 2-opt swap on the route.
    """
    num_cities = len(route)
    i, j = sorted(random.sample(range(num_cities), 2))
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    return new_route

def simulated_annealing(adjacency_matrix, initial_temperature=10000, stopping_temperature=1e-6, num_iterations=10000):
    """
    Solve TSP using Simulated Annealing algorithm with k-opt.
    """
    num_cities = len(adjacency_matrix)

    current_order = list(range(num_cities))
    random.shuffle(current_order)
    current_cost = total_distance(current_order, adjacency_matrix)

    best_order = current_order.copy()
    best_cost = current_cost

    temperature = initial_temperature
    iteration = 0

    while temperature > stopping_temperature and iteration < num_iterations:
        # Generate a new solution
        new_order = two_opt_swap(current_order)
        new_cost = total_distance(new_order, adjacency_matrix)

        # Acceptance probability based on new and current costs
        delta_cost = new_cost - current_cost
        if delta_cost < 0 or random.random() < math.exp(-delta_cost / temperature):
            current_order = new_order
            current_cost = new_cost

        # Keep track of the best solution found so far
        if current_cost < best_cost:
            best_order = current_order.copy()
            best_cost = current_cost

        # Cooling schedule: geometric cooling
        temperature *= 0.995
        iteration += 1

    return best_order, best_cost, total_distance(best_order, adjacency_matrix)



if __name__ == "__main__":
    folder_path = "/home/sgilango/Traveling-Salesman-Problem---AI/TestRuns/"
    file_name = "tsp-problem-100-100-100-5-1.txt"  # Example file name
    file_path = os.path.join(folder_path, file_name)

    num_locations, adjacency_matrix = read_tsp_file(file_path)

    start_time = time.time()
    best_order, best_cost, total_dist = simulated_annealing(adjacency_matrix)
    end_time = time.time()

    print("Best order:", best_order)
    print("Best cost:", best_cost)
    print("Total distance traveled:", total_dist)
    print("Execution time:", end_time - start_time, "seconds")



Best order: [61, 84, 34, 19, 47, 62, 32, 3, 24, 75, 56, 11, 99, 6, 18, 78, 35, 2, 89, 72, 17, 54, 48, 81, 7, 1, 83, 16, 31, 0, 67, 63, 43, 80, 21, 69, 82, 41, 65, 30, 74, 70, 15, 71, 86, 58, 8, 88, 76, 90, 20, 38, 59, 60, 25, 44, 55, 53, 28, 50, 96, 79, 42, 39, 23, 46, 40, 26, 68, 66, 9, 57, 12, 4, 73, 5, 10, 97, 33, 14, 52, 13, 64, 91, 29, 87, 37, 95, 51, 94, 22, 85, 27, 45, 98, 49, 92, 36, 93, 77]
Best cost: 9103.511556202493
Total distance traveled: 9103.511556202493
Execution time: 0.3184642791748047 seconds


In [26]:
#best so far

import numpy as np
import random
import math
import os
import time

def read_tsp_file(filename):
    """
    Read TSP file and extract information.
    """
    with open(filename, 'r') as file:
        lines = file.readlines()
        num_locations = int(lines[0].strip())
        adjacency_matrix = np.zeros((num_locations, num_locations))
        for i in range(num_locations):
            distances = list(map(float, lines[i + 1].strip().split()))
            adjacency_matrix[i] = distances
    return num_locations, adjacency_matrix

def distance(city1, city2):
    """
    Calculate Euclidean distance between two cities.
    """
    return np.linalg.norm(city1 - city2)

def total_distance(cities_order, distances):
    """
    Calculate the total distance of the path.
    """
    total = 0
    num_cities = len(cities_order)
    for i in range(num_cities - 1):
        total += distances[cities_order[i], cities_order[i + 1]]
    total += distances[cities_order[-1], cities_order[0]]  # Return to the starting city
    return total

def two_opt_swap(route, i, j):
    """
    Perform a 2-opt swap on the route.
    """
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    return new_route

def nearest_neighbor_heuristic(distances):
    """
    Initialize the solution using the nearest neighbor heuristic.
    """
    num_cities = len(distances)
    current_city = random.randint(0, num_cities - 1)
    unvisited_cities = set(range(num_cities))
    unvisited_cities.remove(current_city)
    route = [current_city]

    while unvisited_cities:
        nearest_city = min(unvisited_cities, key=lambda city: distances[current_city][city])
        route.append(nearest_city)
        unvisited_cities.remove(nearest_city)
        current_city = nearest_city

    return route

def simulated_annealing(adjacency_matrix, initial_temperature=10000, stopping_temperature=1e-6, num_iterations=10000):
    """
    Solve TSP using Simulated Annealing algorithm with 2-opt.
    """
    num_cities = len(adjacency_matrix)

    current_order = nearest_neighbor_heuristic(adjacency_matrix)
    current_cost = total_distance(current_order, adjacency_matrix)

    best_order = current_order.copy()
    best_cost = current_cost

    temperature = initial_temperature
    iteration = 0

    while temperature > stopping_temperature and iteration < num_iterations:
        # Generate a new solution
        i, j = sorted(random.sample(range(num_cities), 2))
        new_order = two_opt_swap(current_order, i, j)
        new_cost = total_distance(new_order, adjacency_matrix)

        # Acceptance probability based on new and current costs
        delta_cost = new_cost - current_cost
        if delta_cost < 0 or random.random() < math.exp(-delta_cost / temperature):
            current_order = new_order
            current_cost = new_cost

        # Keep track of the best solution found so far
        if current_cost < best_cost:
            best_order = current_order.copy()
            best_cost = current_cost

        # Cooling schedule: geometric cooling
        temperature *= 0.995
        iteration += 1

    return best_order, best_cost, total_distance(best_order, adjacency_matrix)


if __name__ == "__main__":
    folder_path = "/home/sgilango/Traveling-Salesman-Problem---AI/TestRuns/"
    file_name = "tsp-problem-100-100-100-25-1.txt"  # Example file name
    file_path = os.path.join(folder_path, file_name)

    num_locations, adjacency_matrix = read_tsp_file(file_path)

    start_time = time.time()
    best_order, best_cost, total_dist = simulated_annealing(adjacency_matrix)
    end_time = time.time()

    print("Best order:", best_order)
    print("Best cost:", best_cost)
    print("Total distance traveled:", total_dist)
    print("Execution time:", end_time - start_time, "seconds")


Best order: [11, 64, 31, 90, 54, 12, 48, 60, 1, 26, 69, 92, 45, 93, 84, 77, 15, 49, 0, 55, 73, 30, 46, 3, 86, 13, 67, 80, 47, 65, 32, 87, 50, 66, 23, 59, 8, 9, 63, 88, 89, 7, 40, 68, 52, 43, 16, 44, 71, 10, 21, 82, 41, 17, 5, 28, 81, 58, 2, 53, 39, 37, 25, 83, 70, 42, 35, 74, 27, 14, 78, 62, 51, 97, 24, 91, 38, 95, 22, 33, 99, 94, 61, 96, 98, 75, 20, 6, 56, 72, 85, 4, 76, 19, 79, 36, 18, 29, 34, 57]
Best cost: 4395.433655108736
Total distance traveled: 4395.433655108736
Execution time: 0.33534693717956543 seconds


In [27]:
#plots


# Solution Quality Boxplot
plt.figure(figsize=(10, 6))
data.boxplot(column='Best Distance', by='Input', grid=False)
plt.xlabel('TSP instances or parameter settings')
plt.ylabel('Total distance of the best solution')
plt.title('Solution Quality Boxplot')
plt.show()

# Execution Time Boxplot
plt.figure(figsize=(10, 6))
data.boxplot(column='Time', by='Input', grid=False)
plt.xlabel('TSP instances or parameter settings')
plt.ylabel('Execution time')
plt.title('Execution Time Boxplot')
plt.show()

# Scalability Analysis
plt.figure(figsize=(10, 6))
plt.plot(data['Input'], data['Time'], marker='o', linestyle='-')
plt.xlabel('Number of cities in the TSP instance')
plt.ylabel('Execution time')
plt.title('Scalability Analysis')
plt.grid(True)
plt.show()

In [1]:
import numpy as np
import random
import math
import os
import time
import pandas as pd

def read_tsp_file(filename):
    """
    Read TSP file and extract information.
    """
    with open(filename, 'r') as file:
        lines = file.readlines()
        num_locations = int(lines[0].strip())
        adjacency_matrix = np.zeros((num_locations, num_locations))
        for i in range(num_locations):
            distances = list(map(float, lines[i + 1].strip().split()))
            adjacency_matrix[i] = distances
    return num_locations, adjacency_matrix

def distance(city1, city2):
    """
    Calculate Euclidean distance between two cities.
    """
    return np.linalg.norm(city1 - city2)

def total_distance(cities_order, distances):
    """
    Calculate the total distance of the path.
    """
    total = 0
    num_cities = len(cities_order)
    for i in range(num_cities - 1):
        total += distances[cities_order[i], cities_order[i + 1]]
    total += distances[cities_order[-1], cities_order[0]]  # Return to the starting city
    return total

def two_opt_swap(route):
    """
    Perform a 2-opt swap on the route.
    """
    num_cities = len(route)
    i, j = sorted(random.sample(range(num_cities), 2))
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    return new_route

def simulated_annealing(adjacency_matrix, initial_temperature=10000, stopping_temperature=1e-6, num_iterations=10000):
    """
    Solve TSP using Simulated Annealing algorithm with 2-opt.
    """
    num_cities = len(adjacency_matrix)

    current_order = list(range(num_cities))
    random.shuffle(current_order)
    current_cost = total_distance(current_order, adjacency_matrix)

    best_order = current_order.copy()
    best_cost = current_cost

    temperature = initial_temperature
    iteration = 0

    while temperature > stopping_temperature and iteration < num_iterations:
        # Generate a new solution
        new_order = two_opt_swap(current_order)
        new_cost = total_distance(new_order, adjacency_matrix)

        # Acceptance probability based on new and current costs
        delta_cost = new_cost - current_cost
        if delta_cost < 0 or random.random() < math.exp(-delta_cost / temperature):
            current_order = new_order
            current_cost = new_cost

        # Keep track of the best solution found so far
        if current_cost < best_cost:
            best_order = current_order.copy()
            best_cost = current_cost

        # Cooling schedule: geometric cooling
        temperature *= 0.995
        iteration += 1

    return best_order, best_cost, total_distance(best_order, adjacency_matrix)

def solve_tsp_files(folder_path):
    """
    Solve TSP for all .txt files in the given folder.
    """
    results = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            num_locations, adjacency_matrix = read_tsp_file(file_path)
            start_time = time.time()
            best_order, best_cost, total_dist = simulated_annealing(adjacency_matrix)
            end_time = time.time()
            execution_time = end_time - start_time
            results.append([file_name, best_order, best_cost, execution_time])
    return results


In [2]:
folder_path = "/home/sgilango/Traveling-Salesman-Problem---AI/TestRuns"  # Adjust folder path as needed

results = solve_tsp_files(folder_path)
df = pd.DataFrame(results, columns=['input', 'calculated_final_route', 'distance_cost', 'time_taken'])
print(df)

                               input  \
0   tsp-problem-800-100-150-45-1.txt   
1  tsp-problem-500-1000-150-35-1.txt   
2    tsp-problem-700-500-50-25-1.txt   
3   tsp-problem-1000-1000-50-5-1.txt   
4   tsp-problem-500-1000-100-5-1.txt   
5  tsp-problem-300-1000-150-45-1.txt   
6   tsp-problem-500-100-100-25-1.txt   
7  tsp-problem-400-5000-100-45-1.txt   
8   tsp-problem-900-1000-50-25-1.txt   
9   tsp-problem-200-1000-50-25-1.txt   

                              calculated_final_route  distance_cost  \
0  [311, 414, 688, 65, 250, 634, 77, 605, 353, 34...   80922.628653   
1  [152, 43, 327, 75, 311, 351, 161, 450, 354, 24...   51423.033089   
2  [164, 674, 605, 400, 410, 367, 70, 241, 236, 6...   16506.720118   
3  [599, 7, 694, 37, 146, 266, 886, 919, 897, 380...   45148.090116   
4  [245, 186, 200, 489, 378, 198, 487, 253, 282, ...   46832.409915   
5  [217, 104, 179, 258, 71, 195, 66, 160, 162, 18...   23104.805790   
6  [260, 261, 460, 14, 106, 375, 426, 143, 97, 25...   35474.4